In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os

In [2]:
%load_ext sql
%sql postgresql://toofanmacpro@localhost:5432/testdb

In [3]:
def fromDB(query: str) -> pd.DataFrame:
    try:
        engine = create_engine('postgresql://toofanmacpro@localhost:5432/testdb')
        
        with engine.connect() as connection:
            df = pd.read_sql(query, connection)
            print('read success')
        return df
        
    except Exception as e:
        print(f"error connectin {e}")

In [4]:
def toDB(df: pd.DataFrame, tableName: str):
    try: 
        engine = create_engine('postgresql://toofanmacpro@localhost:5432/testdb')
        
        with engine.connect() as connection:
            df.to_sql(tableName, engine, if_exists = 'replace')
            print(f'write success to {tableName} using {engine}')
    except Exception as e:
        print(f"connection failed: {e}")

In [5]:
%%sql
SELECT * FROM "Scores" 
ORDER BY score DESC
LIMIT 5;

 * postgresql://toofanmacpro@localhost:5432/testdb
5 rows affected.


index,id,score
2,3,4.0
4,5,4.0
3,4,3.85
1,2,3.65
5,6,3.65


 reusable code?, lets convert the above TOP N Selection into a function

In [6]:
%%sql
CREATE OR REPLACE FUNCTION TopN( 
    N INT,
    tableName VARCHAR
    )
RETURNS TABLE (
    id INT,
    score FLOAT
) AS
$$
BEGIN
RETURN QUERY(
    SELECT * FROM tableName
    ORDER BY score DESC
    LIMIT N
);
END;
$$ LANGUAGE plpgsql;

 * postgresql://toofanmacpro@localhost:5432/testdb
Done.


[]

In [7]:
%%sql
CREATE OR REPLACE FUNCTION TopN(
    N INT,
    tableName VARCHAR,
    orderByColumn VARCHAR
)
RETURNS TABLE (
    score FLOAT
) AS
$$
BEGIN
    RETURN QUERY EXECUTE format('SELECT DISTINCT %I FROM %I ORDER BY %I DESC LIMIT %L',
                                orderByColumn,
                                tableName,
                                orderByColumn,
                                N
                               );
END;
$$ LANGUAGE plpgsql;

 * postgresql://toofanmacpro@localhost:5432/testdb
Done.


[]

In [8]:
%%sql
SELECT TopN(3,'Scores', 'score')

 * postgresql://toofanmacpro@localhost:5432/testdb
3 rows affected.


topn
4.0
3.85
3.65


»» lets look at much complicated example that uses DECLARE

In [9]:
# Get the current date to simulate CURRENT_DATE from SQL
current_date = pd.Timestamp.now().normalize()

# --- Department Data ---
# Create a list of department dictionaries
departments_data = [
    {'department_id': 1, 'department_name': 'Sales'},
    {'department_id': 2, 'department_name': 'Marketing'},
    {'department_id': 3, 'department_name': 'Engineering'},
    {'department_id': 4, 'department_name': 'Human Resources'},
    {'department_id': 5, 'department_name': 'Finance'},
]

# Create the Department DataFrame
df_departments = pd.DataFrame(departments_data)
print("--- Department DataFrame ---")
print(df_departments)
print("\n")

# --- Employee Data ---
# Create a list of employee dictionaries
employees_data = [
    # Sales Department Employees
    {'employee_name': 'Alice Smith', 'department_name': 'Sales', 'salary': 60000.00, 'hire_date_offset_years': 5},
    {'employee_name': 'Bob Johnson', 'department_name': 'Sales', 'salary': 62000.00, 'hire_date_offset_years': 2},
    {'employee_name': 'Charlie Brown', 'department_name': 'Sales', 'salary': 58000.00, 'hire_date_offset_years': 7},
    {'employee_name': 'Diana Prince', 'department_name': 'Sales', 'salary': 65000.00, 'hire_date_offset_years': 0},

    # Marketing Department Employees
    {'employee_name': 'Eve Davis', 'department_name': 'Marketing', 'salary': 70000.00, 'hire_date_offset_years': 6},
    {'employee_name': 'Frank White', 'department_name': 'Marketing', 'salary': 72000.00, 'hire_date_offset_years': 3},
    {'employee_name': 'Grace Taylor', 'department_name': 'Marketing', 'salary': 68000.00, 'hire_date_offset_years': 1},

    # Engineering Department Employees
    {'employee_name': 'Henry Wilson', 'department_name': 'Engineering', 'salary': 90000.00, 'hire_date_offset_years': 8},
    {'employee_name': 'Ivy Moore', 'department_name': 'Engineering', 'salary': 95000.00, 'hire_date_offset_years': 4},
    {'employee_name': 'Jack Green', 'department_name': 'Engineering', 'salary': 88000.00, 'hire_date_offset_years': 2},
    {'employee_name': 'Karen Hall', 'department_name': 'Engineering', 'salary': 92000.00, 'hire_date_offset_years': 9},

    # Human Resources Department Employees
    {'employee_name': 'Liam King', 'department_name': 'Human Resources', 'salary': 55000.00, 'hire_date_offset_years': 1},
    {'employee_name': 'Mia Wright', 'department_name': 'Human Resources', 'salary': 57000.00, 'hire_date_offset_years': 3},

    # Finance Department Employees
    {'employee_name': 'Noah Scott', 'department_name': 'Finance', 'salary': 80000.00, 'hire_date_offset_years': 10},
    {'employee_name': 'Olivia Adams', 'department_name': 'Finance', 'salary': 83000.00, 'hire_date_offset_years': 5}
]

# Create the Employee DataFrame
df_employees = pd.DataFrame(employees_data)

# Map department names to department_ids for the employee DataFrame
# This simulates the JOIN operation in the SQL function
department_id_map = df_departments.set_index('department_name')['department_id'].to_dict()
df_employees['department_id'] = df_employees['department_name'].map(department_id_map)

# Calculate actual hire_date based on current_date and offset years
df_employees['hire_date'] = df_employees['hire_date_offset_years'].apply(
    lambda years: current_date - pd.DateOffset(years=years)
)

# Drop the temporary 'department_name' and 'hire_date_offset_years' columns if not needed
df_employees = df_employees.drop(columns=['department_name', 'hire_date_offset_years'])

# Reorder columns to match the SQL table structure (optional, for clarity)
df_employees = df_employees[['employee_name', 'department_id', 'salary', 'hire_date']]

print("--- Employee DataFrame ---")
print(df_employees)

--- Department DataFrame ---
   department_id  department_name
0              1            Sales
1              2        Marketing
2              3      Engineering
3              4  Human Resources
4              5          Finance


--- Employee DataFrame ---
    employee_name  department_id   salary  hire_date
0     Alice Smith              1  60000.0 2020-07-21
1     Bob Johnson              1  62000.0 2023-07-21
2   Charlie Brown              1  58000.0 2018-07-21
3    Diana Prince              1  65000.0 2025-07-21
4       Eve Davis              2  70000.0 2019-07-21
5     Frank White              2  72000.0 2022-07-21
6    Grace Taylor              2  68000.0 2024-07-21
7    Henry Wilson              3  90000.0 2017-07-21
8       Ivy Moore              3  95000.0 2021-07-21
9      Jack Green              3  88000.0 2023-07-21
10     Karen Hall              3  92000.0 2016-07-21
11      Liam King              4  55000.0 2024-07-21
12     Mia Wright              4  57000.0 2022-07

In [10]:
toDB(df_employees, 'empRecords')

write success to empRecords using Engine(postgresql://toofanmacpro@localhost:5432/testdb)


In [11]:
toDB(df_departments, 'departments')

write success to departments using Engine(postgresql://toofanmacpro@localhost:5432/testdb)


In [12]:
%%sql
SELECT * FROM "empRecords"

 * postgresql://toofanmacpro@localhost:5432/testdb
15 rows affected.


index,employee_name,department_id,salary,hire_date
0,Alice Smith,1,60000.0,2020-07-21 00:00:00
1,Bob Johnson,1,62000.0,2023-07-21 00:00:00
2,Charlie Brown,1,58000.0,2018-07-21 00:00:00
3,Diana Prince,1,65000.0,2025-07-21 00:00:00
4,Eve Davis,2,70000.0,2019-07-21 00:00:00
5,Frank White,2,72000.0,2022-07-21 00:00:00
6,Grace Taylor,2,68000.0,2024-07-21 00:00:00
7,Henry Wilson,3,90000.0,2017-07-21 00:00:00
8,Ivy Moore,3,95000.0,2021-07-21 00:00:00
9,Jack Green,3,88000.0,2023-07-21 00:00:00


In [13]:
%%sql
SELECT * FROM "departments"

 * postgresql://toofanmacpro@localhost:5432/testdb
5 rows affected.


index,department_id,department_name
0,1,Sales
1,2,Marketing
2,3,Engineering
3,4,Human Resources
4,5,Finance


In [14]:
%%sql
CREATE OR REPLACE FUNCTION DeptAvgSalary (                                                      -- passing department name, minimum years of experience, & whether bonus be included in avg salary calculation
    depName VARCHAR,                                                            
    minYears INT,
    inclBonus BOOLEAN DEFAULT FALSE
)
RETURNS TABLE (                                                                                 -- returns average salary & total no of people in the deparment that meet the criteria set in input params
    deptName VARCHAR,
    avgSalary NUMERIC,
    totEmpl BIGINT
) AS 
$$
DECLARE                                                                                         -- declaring the local variables I require rather than hardcoding them into query for easy maintenance
    bonusPct NUMERIC := 0.10;
    totalSal NUMERIC := 0;
    emplCnt BIGINT := 0;
    avgSal NUMERIC;
BEGIN
    IF minYears < 0 THEN                                                                        -- little validation before I dive-in 
        RAISE EXCEPTION 'Min years of exp cannot be negative';
    END IF;
    
    SELECT                                                                                      
        SUM(                                                                                    -- we add up salary, and bonus to it if function params indicate so
            e.salary + CASE WHEN inclBonus THEN e.salary * bonusPct ELSE 0 END
        ),
        COUNT(e.index)                                                                          -- count the employees in the filtered criteria
    INTO 
        totalSal,
        emplCnt
    FROM 
        "empRecords" AS e
    JOIN
        "departments" AS d ON e.department_id = d.department_id
    WHERE
        d.department_name = depName
        AND
        EXTRACT(YEAR FROM AGE(CURRENT_DATE, e.hire_date)) >= minYears;
        
    IF emplCNT >0 THEN 
        avgSalary := totalSal / emplCnt;
    ELSE
        avgSal := 0;    
    END IF;
    
    RETURN QUERY SELECT
        depName AS department_name,
        ROUND(avgSalary) AS average_salary,
        emplCnt AS total_eligible_employees;   
END;
$$ LANGUAGE plpgsql;

 * postgresql://toofanmacpro@localhost:5432/testdb
Done.


[]

In [15]:
%%sql
SELECT * FROM DeptAvgSalary('Sales', 4, FALSE);

 * postgresql://toofanmacpro@localhost:5432/testdb
1 rows affected.


deptname,avgsalary,totempl
Sales,59000,2


»» if the function needs to be dropped

In [17]:
%%sql
SELECT * FROM "departments"

 * postgresql://toofanmacpro@localhost:5432/testdb
5 rows affected.


index,department_id,department_name
0,1,Sales
1,2,Marketing
2,3,Engineering
3,4,Human Resources
4,5,Finance


In [18]:
%%sql
SELECT * FROM "empRecords"

 * postgresql://toofanmacpro@localhost:5432/testdb
15 rows affected.


index,employee_name,department_id,salary,hire_date
0,Alice Smith,1,60000.0,2020-07-21 00:00:00
1,Bob Johnson,1,62000.0,2023-07-21 00:00:00
2,Charlie Brown,1,58000.0,2018-07-21 00:00:00
3,Diana Prince,1,65000.0,2025-07-21 00:00:00
4,Eve Davis,2,70000.0,2019-07-21 00:00:00
5,Frank White,2,72000.0,2022-07-21 00:00:00
6,Grace Taylor,2,68000.0,2024-07-21 00:00:00
7,Henry Wilson,3,90000.0,2017-07-21 00:00:00
8,Ivy Moore,3,95000.0,2021-07-21 00:00:00
9,Jack Green,3,88000.0,2023-07-21 00:00:00


In [16]:
%%sql
DROP FUNCTION TopN(
    N INT,
    tableName VARCHAR,
    orderByColumn VARCHAR
)

 * postgresql://toofanmacpro@localhost:5432/testdb
Done.


[]